In [ ]:
import torch
from pyannote.database import get_protocol, FileFinder

preprocessors = {'audio': FileFinder()}
protocol = get_protocol('VOXCON.SpeakerDiarization.Sample', preprocessors=preprocessors)

diarization_pipeline = torch.hub.load('pyannote/pyannote-audio', 'dia_dihard')

In [ ]:
import time
from pyannote.metrics.diarization import DiarizationErrorRate, JaccardErrorRate

ders = []
jers = []
hypotheses = []

derMetric = DiarizationErrorRate(collar=0.25)
jerMetric = JaccardErrorRate(collar=0.25)

for file in protocol.test():
    hypothesis = diarization_pipeline(file)
    hypotheses.append(hypothesis)
    
    reference = file["annotation"]
    uem = file['annotated']
    der = derMetric(reference, hypothesis, uem)['diarization error rate']
    jer = jerMetric(reference, hypothesis, uem)['diarization error rate']
    ders.append(der)
    jers.append(jer)
    
    uri = file['uri']
    print(f'{uri} DER = {100 * der:.1f}% JER = {100 * jer:.1f}% {time.strftime("%H:%M:%S")}')

In [ ]:
metric = DiarizationErrorRate(collar=0.25)

# der['diarization error rate'] = (der['false alarm'] + der['missed detection'] + der['confusion']) / der['total']

i = 0
for file in protocol.test():
    hypotesis = diarizations[i]
    reference = file["annotation"]
    uem = file['annotated']
    der = metric(reference, hypotesis)
    print(der)
    i+=1

In [ ]:
# Compute global diarization error rate and confidence interval

global_value = abs(metric)
mean, (lower, upper) = metric.confidence_interval()

print(f'DER_total = {100 * global_value:.1f}% mean = {100 * mean:.1f}%')
print(f'lower = {100 * lower:.1f}% upper = {100 * upper:.1f}%')

In [ ]:
for file in protocol.test():
    der = derMetric(groundtruth, diarization)

In [ ]:
list(protocol.test())[7]["annotation"]

In [ ]:
ders

In [ ]:
touch ./lists/voxconverse/sample.der

In [ ]:
derPath = "/home/jovyan/work/uis-rnn-voxsrc20/lists/voxconverse/sample.der"
with open(derPath, "w") as derFile:
    derFile.write("\n".join(ders))

In [ ]:
from pyannote.database.util import load_rttm
rttm = load_rttm('/home/jovyan/work/uis-rnn-voxsrc20/lists/voxconverse/sample.rttm')[test_file['uri']]
test_file['uri']

In [ ]:
from pyannote.audio.features import RawAudio
from pyannote.core import Segment, notebook
from IPython.display import Audio

# make notebook visualization zoom on 600s < t < 660s time range
# EXCERPT = Segment(600, 660)
# notebook.crop = EXCERPT

# visualize excerpt groundtruth
groundtruth

file = {'uri': test_file['uri'], 'audio': test_file['audio']}

# load audio waveform, crop excerpt, and play it
waveform = RawAudio(sample_rate=16000)(file).data
Audio(data=waveform.squeeze(), rate=16000, autoplay=True)

In [ ]:
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f'Speaker "{speaker}" speaks between t={turn.start:.1f}s and t={turn.end:.1f}s.')

mapping = metric.optimal_mapping(groundtruth, diarization)
diarization.rename_labels(mapping=mapping)